In [ ]:
#Install the below set of libraries of langchain if not installed alredy
#!pip install langchain_community langchainhub chromadb langchain langchain-openai

In [ ]:
#!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

#Load environment variables from .env file
load_dotenv()

#Access the Openai key stored in .env file
open_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
import openai
openai.api_key = open_api_key

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=["https://www.educosys.com/course/genai"])

docs = loader.load()
print(docs)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

print(splits[0])
print(splits[1])

In [ ]:
print(len(splits))

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding= OpenAIEmbeddings())

In [ ]:
print(vectorstore._collection.count())

In [ ]:
retriever =  vectorstore.as_retriever()

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
#Setup LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(docs):
    return ("\n".join(doc.page_content for doc in docs))

rag_chain = ({"context": retriever | format_docs,"question": RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [ ]:
rag_chain.invoke("Are there recordings of the course available?")

In [ ]:
rag_chain.invoke("Are there testimonials of the course available?")

In [ ]:
rag_chain.invoke("Are there recordings of the course available? Name of the students who have shared the their testimonials")